<a href="https://colab.research.google.com/github/MarVinikoi/DL_NafldReserch/blob/main/NAFLD_TCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras-tcn
!pip install scikit-learn
!pip install livelossplot
!pip install visualkeras
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.7/993.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.6/230.6 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 10.5 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd "/content/drive/MyDrive/PesquisaNAFLD/Data/NovaBase/"

/content/drive/MyDrive/PesquisaNAFLD/Data/NovaBase


In [4]:
import cv2
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import glob, os
import math
import random
import optuna
from tensorflow.keras import callbacks
from tcn import TCN, tcn_full_summary
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from google.colab.patches import cv2_imshow

In [5]:
#Variáveis Globais

path_doentes = "images/doentes"
path_saudaveis = "images/nao_doentes"

In [6]:
#Carregando as imagens


labels_treino = []
labels_teste = []
labels_val = []


vetor_imgs_doe = []
vetor_imgs_sau = []
vetor_final_imgs = []

#Carregando os vetores de imagens e os transformando em séries temporais
#Vetor de Doentes
for paciente in os.listdir(path_doentes):
  path_paciente = os.path.join(path_doentes, paciente)
  print(path_paciente)
  paths_imgs = sorted(glob.glob(os.path.join(path_paciente, "*.jpg")))
  for i, img_path in enumerate(paths_imgs):
    #Se a imagem for a primeira, última ou do meio, iremos operar sobre ela
    if(img_path == paths_imgs[0] or img_path == paths_imgs[-1] or img_path == paths_imgs[round((len(paths_imgs)-1)/2)]):
      image = cv2.imread(img_path)
      print(f"image {i} = {img_path}")

      vetor_imgs_doe.append(image)

#LABELS --> 0 = DOENTES  ||  1 = SAUDÁVEIS
#labels.extend(np.zeros(len(vetor_final_imgs)/timesteps), dtype = "int32")

#Vetor de Saudáveis
for paciente in os.listdir(path_saudaveis):
  path_paciente = os.path.join(path_saudaveis, paciente)
  paths_imgs = sorted(glob.glob(os.path.join(path_paciente, "*.jpg")))
  print(path_paciente)
  for i, img_path in enumerate(paths_imgs):
    #Se a imagem for a primeira, última ou do meio, iremos operar sobre ela
    if(img_path == paths_imgs[0] or img_path == paths_imgs[-1] or img_path == paths_imgs[round((len(paths_imgs)-1)/2)]):

      image = cv2.imread(img_path)
      vetor_imgs_sau.append(image)
      print(f"image {i} = {img_path}")


#LABELS --> 0 = DOENTES  ||  1 = SAUDÁVEIS
#labels.extend(np.ones(len(vetor_final_imgs) - len(labels)))



print(f"Len vetor final de doentes: {len(vetor_imgs_doe)}")
print(f"Len vetor final de saudáveis: {len(vetor_imgs_sau)}")

images/doentes/doente_1
image 0 = images/doentes/doente_1/table1.jpg
image 2 = images/doentes/doente_1/table3.jpg
image 4 = images/doentes/doente_1/table5.jpg
images/doentes/doente_2
image 0 = images/doentes/doente_2/table1.jpg
image 2 = images/doentes/doente_2/table3.jpg
image 4 = images/doentes/doente_2/table5.jpg
images/doentes/doente_3
image 0 = images/doentes/doente_3/table1.jpg
image 2 = images/doentes/doente_3/table3.jpg
image 4 = images/doentes/doente_3/table5.jpg
images/doentes/doente_4
image 0 = images/doentes/doente_4/table1.jpg
image 2 = images/doentes/doente_4/table3.jpg
image 4 = images/doentes/doente_4/table5.jpg
images/doentes/doente_5
image 0 = images/doentes/doente_5/table1.jpg
image 2 = images/doentes/doente_5/table3.jpg
image 4 = images/doentes/doente_5/table5.jpg
images/doentes/doente_6
image 0 = images/doentes/doente_6/table1.jpg
image 2 = images/doentes/doente_6/table3.jpg
image 4 = images/doentes/doente_6/table5.jpg
images/doentes/doente_7
image 0 = images/doent

In [7]:
def dataAugmentation(v_treino_doe : list, v_treino_sau):
  labels = []
  treino = []
  lenght_doe = len(v_treino_doe)
  lenght_sau = len(v_treino_sau)
  for i in range(lenght_doe):
    v_treino_doe.append(cv2.flip(v_treino_doe[i], 0))
  for i in range(lenght_doe):
    v_treino_doe.append(cv2.flip(v_treino_doe[i], 1))
  for i in range(lenght_doe):
    v_treino_doe.append(cv2.rotate(v_treino_doe[i], cv2.ROTATE_90_CLOCKWISE))

  for i in range(lenght_sau):
    v_treino_sau.append(cv2.flip(v_treino_sau[i], 0))
  for i in range(lenght_sau):
    v_treino_sau.append(cv2.flip(v_treino_sau[i], 1))
  for i in range(lenght_sau):
    v_treino_sau.append(cv2.rotate(v_treino_sau[i], cv2.ROTATE_90_CLOCKWISE))

  labels = [0 for x in range(len(v_treino_doe))]
  labels.extend(1 for x in range(len(v_treino_sau)))

  treino.extend(v_treino_doe)
  treino.extend(v_treino_sau)

  return (treino, labels)



treino_doe = []
treino_sau = []
teste = []
val = []

#Utilizando cerca de 60% das imagens para treino
treino_doe.extend(vetor_imgs_doe[:round(len(vetor_imgs_doe)*0.6)])
treino_sau.extend(vetor_imgs_sau[:round(len(vetor_imgs_sau)*0.6)])

treino, labels_treino = dataAugmentation(treino_doe, treino_sau)

#Utilizando cerca de 20% das imagens para teste
teste.extend(vetor_imgs_doe[round(len(vetor_imgs_doe)*0.6):round(len(vetor_imgs_doe)*0.8)])
labels_teste = [0 for x in range(len(teste))]
teste.extend(vetor_imgs_sau[round(len(vetor_imgs_sau)*0.6):round(len(vetor_imgs_sau)*0.8)])
labels_teste.extend(1 for x in range(len(teste)-len(labels_teste)))


#Utilizando cerca de 20% das imagens para validação
val.extend(vetor_imgs_doe[round(len(vetor_imgs_doe)*0.8):])
labels_val = [0 for x in range(len(val))]
val.extend(vetor_imgs_sau[round(len(vetor_imgs_sau)*0.8):])
labels_val.extend(1 for x in range(len(val)-len(labels_val)))



In [8]:
#Conversão das imagens em séries temporais

def toTemporal(v_treino, v_teste, v_val, labels_treino, labels_teste, labels_val):
  v_final_treino = []
  v_final_teste = []
  v_final_val = []
  labels_final_treino = []
  labels_final_teste = []
  labels_final_val = []
  v_imgs = []

  for i, elm in enumerate(v_treino):

    r_img, g_img, b_img = cv2.split(elm)

    r_img = np.array(r_img)
    r_img = np.ndarray.flatten(r_img)

    v_imgs.append(r_img)

    if len(v_imgs) == 3:
      v_final_treino.append((v_imgs[0], v_imgs[1], v_imgs[2]))
      v_imgs.clear()


  for elm in v_teste:

    r_img, g_img, b_img = cv2.split(elm)

    r_img = np.array(r_img)
    r_img = np.ndarray.flatten(r_img)

    v_imgs.append(r_img)

    if len(v_imgs) == 3:
      v_final_teste.append((v_imgs[0], v_imgs[1], v_imgs[2]))
      v_imgs.clear()


  for elm in v_val:

    r_img, g_img, b_img = cv2.split(elm)

    r_img = np.array(r_img)
    r_img = np.ndarray.flatten(r_img)

    v_imgs.append(r_img)

    if len(v_imgs) == 3:
      # plot_3d(v_imgs[0], v_imgs[1], v_imgs[2])
      v_final_val.append((v_imgs[0], v_imgs[1], v_imgs[2]))
      v_imgs.clear()

  for i in range(0,len(labels_treino),3):
    labels_final_treino.append(labels_treino[i])

  for i in range(0,len(labels_teste),3):
    labels_final_teste.append(labels_teste[i])

  for i in range(0,len(labels_val),3):
    labels_final_val.append(labels_val[i])

  print("Estado dos conjuntos:")
  print(f"Treino: {len(v_final_treino)} -- Labels: {len(labels_final_treino)}")
  print(f"Teste: {len(v_final_teste)} -- Labels: {len(labels_final_teste)}")
  print(f"Val: {len(v_final_val)} -- Labels: {len(labels_final_val)}")

  return (v_final_treino, v_final_teste, v_final_val, labels_final_treino, labels_final_teste, labels_final_val)

In [9]:
treino, teste, val, labels_treino, labels_teste, labels_val = toTemporal(treino, teste, val, labels_treino, labels_teste, labels_val)

treino = np.array(treino)
teste = np.array(teste)
val = np.array(val)

print(treino.shape)
print(teste.shape)
print(val.shape)

Estado dos conjuntos:
Treino: 140 -- Labels: 140
Teste: 11 -- Labels: 12
Val: 12 -- Labels: 12
(140, 3, 80089)
(11, 3, 80089)
(12, 3, 80089)


In [10]:
#Embaralhando os vetores de imagens térmicas

random.seed(8528741)

zipped_treino = list(zip(treino, labels_treino))
zipped_teste = list(zip(teste, labels_teste))
zipped_val = list(zip(val, labels_val))

random.shuffle(zipped_treino)
random.shuffle(zipped_teste)
random.shuffle(zipped_val)

treino, labels_treino = zip(*zipped_treino)
teste, labels_teste = zip(*zipped_teste)
val, labels_val = zip(*zipped_val)

treino = np.array(treino)
teste = np.array(teste)
val = np.array(val)
labels_treino = np.array(labels_treino)
labels_teste = np.array(labels_teste)
labels_val = np.array(labels_val)

In [18]:
#Otimização com Optuna

from sklearn.metrics import classification_report, confusion_matrix, recall_score, precision_score, f1_score

#Acumuladores de Métricas
recall = []
prec = []
f1 = []
matrizes = []

n_iter = 10

callback_funcs = []


def objective(trial):

  tcn_nbfilters = trial.suggest_categorical("nb_filters", [16, 32, 64, 128])
  tcn_kernelsize = trial.suggest_int("kernel_size", 3, 9, log=True)
  tcn_nbstacks = trial.suggest_int("nb_stacks", 1, 3, log=True)
  tcn_kernelinit = trial.suggest_categorical("kernel_initializer", ["he_normal", "glorot_uniform"])
  #tcn_dropout = trial.suggest_categorical("dropout_rate", [0.001, 0.01, 0.1])

  model = Sequential([
      TCN(
          nb_filters = tcn_nbfilters, kernel_size = tcn_kernelsize, nb_stacks = tcn_nbstacks,
          kernel_initializer = tcn_kernelinit, dilations = (1,2,4,8,16,32,64)
          ),
      tf.keras.layers.Dense(units=1,activation="sigmoid")

  ])

  model.compile(optimizer="adam", loss = tf.losses.BinaryFocalCrossentropy(), metrics = ["accuracy"])
  print("\n\n")

  hist = model.fit(treino, labels_treino, validation_data=(val, labels_val), epochs=20, callbacks = callback_funcs)

  y_pred = model.predict(teste)
  for i, elm in enumerate(y_pred):
    y_pred[i] = np.round(elm)
  y_true = labels_teste

  recall.append(recall_score(y_true, y_pred, average='weighted'))
  prec.append(precision_score(y_true, y_pred, average='weighted'))
  f1.append(f1_score(y_true, y_pred, average='weighted'))
  matrix = confusion_matrix(y_true, y_pred)
  matrizes.append(matrix)

  return precision_score(y_true, y_pred, average='weighted')


study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=n_iter)


recall_media = np.asarray(recall).mean(axis=0)
prec_media = np.asarray(prec).mean(axis=0)
f1_media = np.asarray(f1).mean(axis=0)

#calculando os desvios das execuções
recall_std = np.asarray(recall).std(axis=0)
prec_std = np.asarray(prec).std(axis=0)
f1_std = np.asarray(f1).std(axis=0)

#sns.heatmap(matrix, cmap='Blues', annot=True)

print ("Recall: " + str(recall_media) + " +- " + str(recall_std))
print ("Precisão: " + str(prec_media) + " +- " + str(prec_std))
print ("F1: " + str(f1_media) + " +- " + str(f1_std))




[I 2023-12-18 16:43:14,043] A new study created in memory with name: no-name-2384227a-01aa-456e-9dd0-ad7c4f8ad321





Epoch 1/30
5/5 [==============================] - 59s 11s/step - loss: 3645.1821 - accuracy: 0.5214 - val_loss: 3028.1985 - val_accuracy: 0.6667
Epoch 2/30
5/5 [==============================] - 1s 134ms/step - loss: 2096.8250 - accuracy: 0.5214 - val_loss: 36.5858 - val_accuracy: 0.8333
Epoch 3/30
5/5 [==============================] - 1s 142ms/step - loss: 795.2663 - accuracy: 0.7000 - val_loss: 825.3898 - val_accuracy: 0.6667
Epoch 4/30
5/5 [==============================] - 1s 162ms/step - loss: 372.6501 - accuracy: 0.6643 - val_loss: 344.8022 - val_accuracy: 0.7500
Epoch 5/30
5/5 [==============================] - 1s 140ms/step - loss: 129.6324 - accuracy: 0.7571 - val_loss: 40.5321 - val_accuracy: 0.8333
Epoch 6/30
5/5 [==============================] - 1s 144ms/step - loss: 112.2299 - accuracy: 0.8071 - val_loss: 35.6304 - val_accuracy: 0.9167
Epoch 7/30
5/5 [==============================] - 1s 161ms/step - loss: 68.1023 - accuracy: 0.8000 - val_loss: 104.2834 - val_accuracy

[I 2023-12-18 16:44:44,983] Trial 0 finished with value: 0.9191919191919191 and parameters: {'nb_filters': 64, 'kernel_size': 7, 'nb_stacks': 2, 'kernel_initializer': 'glorot_uniform'}. Best is trial 0 with value: 0.9191919191919191.





Epoch 1/30
5/5 [==============================] - 17s 777ms/step - loss: 3114.4287 - accuracy: 0.5357 - val_loss: 2589.6726 - val_accuracy: 0.6667
Epoch 2/30
5/5 [==============================] - 0s 78ms/step - loss: 1414.2028 - accuracy: 0.5714 - val_loss: 1213.8068 - val_accuracy: 0.6667
Epoch 3/30
5/5 [==============================] - 0s 77ms/step - loss: 690.5637 - accuracy: 0.6714 - val_loss: 1305.0360 - val_accuracy: 0.6667
Epoch 4/30
5/5 [==============================] - 0s 84ms/step - loss: 520.1569 - accuracy: 0.7429 - val_loss: 93.8541 - val_accuracy: 0.7500
Epoch 5/30
5/5 [==============================] - 0s 78ms/step - loss: 202.3175 - accuracy: 0.8500 - val_loss: 375.3274 - val_accuracy: 0.6667
Epoch 6/30
5/5 [==============================] - 0s 84ms/step - loss: 127.0914 - accuracy: 0.8643 - val_loss: 250.3722 - val_accuracy: 0.6667
Epoch 7/30
5/5 [==============================] - 0s 81ms/step - loss: 81.9279 - accuracy: 0.8786 - val_loss: 345.3737 - val_accuracy

[I 2023-12-18 16:45:35,634] Trial 1 finished with value: 0.9191919191919191 and parameters: {'nb_filters': 32, 'kernel_size': 5, 'nb_stacks': 2, 'kernel_initializer': 'he_normal'}. Best is trial 0 with value: 0.9191919191919191.


...




Epoch 1/30
5/5 [==============================] - 14s 335ms/step - loss: 2315.0928 - accuracy: 0.4786 - val_loss: 404.1917 - val_accuracy: 0.6667
Epoch 2/30
5/5 [==============================] - 1s 115ms/step - loss: 1269.5387 - accuracy: 0.5286 - val_loss: 4109.3618 - val_accuracy: 0.6667
Epoch 3/30
5/5 [==============================] - 1s 114ms/step - loss: 2015.9447 - accuracy: 0.7286 - val_loss: 415.1068 - val_accuracy: 0.6667
Epoch 4/30
5/5 [==============================] - 1s 118ms/step - loss: 810.7759 - accuracy: 0.6714 - val_loss: 1323.5098 - val_accuracy: 0.6667
Epoch 5/30
5/5 [==============================] - 1s 112ms/step - loss: 453.8972 - accuracy: 0.7571 - val_loss: 73.0450 - val_accuracy: 0.8333
Epoch 6/30
5/5 [==============================] - 1s 114ms/step - loss: 173.5879 - accuracy: 0.8500 - val_loss: 330.9795 - val_accuracy: 0.8333
Epoch 7/30
5/5 [==============================] - 1s 112ms/step - loss: 60.0804 - accuracy: 0.8643 - val_loss: 255.4371 - val_ac

[I 2023-12-18 16:46:52,340] Trial 3 finished with value: 1.0 and parameters: {'nb_filters': 64, 'kernel_size': 6, 'nb_stacks': 2, 'kernel_initializer': 'glorot_uniform'}. Best is trial 2 with value: 1.0.





Epoch 1/30
5/5 [==============================] - 10s 605ms/step - loss: 8415.5742 - accuracy: 0.4786 - val_loss: 6333.0063 - val_accuracy: 0.6667
Epoch 2/30
5/5 [==============================] - 1s 106ms/step - loss: 5619.7373 - accuracy: 0.7214 - val_loss: 12.0524 - val_accuracy: 0.9167
Epoch 3/30
5/5 [==============================] - 1s 106ms/step - loss: 1283.2816 - accuracy: 0.6357 - val_loss: 2114.6331 - val_accuracy: 0.6667
Epoch 4/30
5/5 [==============================] - 1s 109ms/step - loss: 1101.3065 - accuracy: 0.6286 - val_loss: 426.2914 - val_accuracy: 0.7500
Epoch 5/30
5/5 [==============================] - 0s 99ms/step - loss: 550.3024 - accuracy: 0.7929 - val_loss: 182.4148 - val_accuracy: 0.9167
Epoch 6/30
5/5 [==============================] - 0s 93ms/step - loss: 270.5333 - accuracy: 0.8071 - val_loss: 242.9438 - val_accuracy: 0.8333
Epoch 7/30
5/5 [==============================] - 0s 93ms/step - loss: 244.2899 - accuracy: 0.8500 - val_loss: 288.0920 - val_acc

[I 2023-12-18 16:47:17,945] Trial 4 finished with value: 0.9191919191919191 and parameters: {'nb_filters': 128, 'kernel_size': 3, 'nb_stacks': 1, 'kernel_initializer': 'glorot_uniform'}. Best is trial 2 with value: 1.0.


Recall: 0.9454545454545455 +- 0.04453617714151235
Precisão: 0.9515151515151515 +- 0.039587713014677665
F1: 0.9416042780748664 +- 0.04767990729267789


In [18]:
#Teste com os melhores hiperparâmetros obtidos na etapa anterior
from sklearn.metrics import classification_report, confusion_matrix, recall_score, precision_score, f1_score, accuracy_score
from livelossplot import PlotLossesKeras

holdout_iter = 5
recall = []
accuracy = []
prec = []
f1 = []
matrizes = []


callback_funcs = []

for _ in range(holdout_iter):

  model = Sequential([
      TCN(
          nb_filters = 16, kernel_size = 5, nb_stacks = 1,
          kernel_initializer = 'he_normal', dilations = (1,2,4,8,16,32,64)
          ),
      tf.keras.layers.Dense(units=1,activation="sigmoid")

  ])


  model.compile(optimizer="adam", loss = tf.losses.BinaryFocalCrossentropy(), metrics = ["accuracy"])

  hist = model.fit(treino, labels_treino, validation_data=(val, labels_val), epochs=20, callbacks = callback_funcs)

  y_pred = model.predict(teste)
  for i, elm in enumerate(y_pred):
    y_pred[i] = np.round(elm)
  y_true = labels_teste

  recall.append(recall_score(y_true, y_pred, average='weighted'))
  prec.append(precision_score(y_true, y_pred, average='weighted'))
  f1.append(f1_score(y_true, y_pred, average='weighted'))
  accuracy.append(accuracy_score(y_true, y_pred))
  matrix = confusion_matrix(y_true, y_pred)
  matrizes.append(matrix)

recall_media = np.asarray(recall).mean(axis=0)
prec_media = np.asarray(prec).mean(axis=0)
f1_media = np.asarray(f1).mean(axis=0)
accuracy_media = np.asarray(accuracy).mean(axis=0)

#calculando os desvios das execuções
recall_std = np.asarray(recall).std(axis=0)
prec_std = np.asarray(prec).std(axis=0)
f1_std = np.asarray(f1).std(axis=0)
accuracy_std = np.asarray(accuracy).std(axis=0)


print ("Acurácia: " + str(accuracy_media) + " +- " + str(accuracy_std))
print ("Recall: " + str(recall_media) + " +- " + str(recall_std))
print ("Precisão: " + str(prec_media) + " +- " + str(prec_std))
print ("F1: " + str(f1_media) + " +- " + str(f1_std))

Epoch 1/20
5/5 [==============================] - 6s 210ms/step - loss: 1676.1151 - accuracy: 0.3857 - val_loss: 1735.2958 - val_accuracy: 0.6667
Epoch 2/20
5/5 [==============================] - 0s 62ms/step - loss: 946.5355 - accuracy: 0.5929 - val_loss: 61.0603 - val_accuracy: 0.9167
Epoch 3/20
5/5 [==============================] - 0s 65ms/step - loss: 390.1703 - accuracy: 0.7429 - val_loss: 57.2763 - val_accuracy: 0.8333
Epoch 4/20
5/5 [==============================] - 0s 58ms/step - loss: 278.7560 - accuracy: 0.7000 - val_loss: 338.1283 - val_accuracy: 0.7500
Epoch 5/20
5/5 [==============================] - 0s 66ms/step - loss: 248.8253 - accuracy: 0.8000 - val_loss: 22.8721 - val_accuracy: 0.9167
Epoch 6/20
5/5 [==============================] - 0s 67ms/step - loss: 204.8860 - accuracy: 0.8000 - val_loss: 254.4404 - val_accuracy: 0.7500
Epoch 7/20
5/5 [==============================] - 0s 59ms/step - loss: 102.7372 - accuracy: 0.8500 - val_loss: 53.1019 - val_accuracy: 0.8333
